In [1]:
import pandas as pd
import math
import numpy as np
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, FactorRange, Legend, HoverTool, Label
from bokeh.transform import dodge
import bokeh.palettes
output_notebook()
# Standard plotly imports
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import os
from sklearn.linear_model import LinearRegression

Loading BokehJS ...

In [2]:
path = '/Users/tuethomsen28/Google Drev/SocialDataVizz/Data'

focus_municipality = ["Albertslund","Ballerup","Brøndby","Dragør","Frederiksberg","Gentofte","Gladsaxe",
                           "Glostrup","Herlev","Hvidovre","Høje-Taastrup","Ishøj","København",
                           "Rødovre","Tårnby","Vallensbæk"]

#df_speed_feb = pd.read_csv(os.path.join(path,'Fremkommelighedsanalyse_med_fraktil_og_potentialeHast_Feb.csv'),sep=";",
#                 header= 2,index_col=False,decimal=",")

#df_speed_mar_apr = pd.read_csv(os.path.join(path,'Fremkommelighedsanalyse_med_fraktil_og_potentiale_Hast_MarApr.csv'),sep=";",
#                 header= 2,index_col=False,decimal=",")

df_passenger = pd.read_csv(os.path.join(path,"passenger_decrease.csv"))

df_age_distribution = pd.read_excel(os.path.join(path,'Data_aldersfordeling.xlsx'))

df_unemployment = pd.read_excel(os.path.join(path,'Data_fuldtidsledige.xlsx'))
df_unemployment = df_unemployment[df_unemployment.Municipality.isin(focus_municipality)].set_index("Municipality").loc[focus_municipality]

df_education = pd.read_csv(os.path.join(path,"Uddannelsesniveau.csv"),sep=";",index_col=0)
df_education = df_education.loc[focus_municipality]
df_pct_high_education = df_education.copy()
df_pct_high_education = pd.DataFrame(df_pct_high_education.iloc[:,6:].sum(1) / df_pct_high_education.sum(1),
                                     columns = ["pct_highly_educated"])

df_ethnicity = pd.read_csv(os.path.join(path,"ethnicity.csv"),sep=";",index_col=0)

df_pct_ethnicity = df_ethnicity[["Indvandrere fra ikke-vestlige lande","Efterkommere fra ikke-vestlige lande"]].sum(1)/df_ethnicity[
    ["Personer med dansk oprindelse","Indvandrere fra vestlige lande","Efterkommere fra vestlige lande"]].sum(1)

df_pct_ethnicity = pd.DataFrame(df_pct_ethnicity,columns=["pct_not_vestern"])

df_pct_ethnicity = df_pct_ethnicity.loc[focus_municipality]

df_income = pd.read_excel(os.path.join(path,'Income2.xlsx'))
df_income.columns = ["Kommune","Income"]
df_income = df_income.set_index("Kommune")
df_income = df_income.loc[focus_municipality]

df_covid = pd.read_csv(os.path.join(path,"Corona_hosp_1603_2204.csv"),index_col=0)
df_covid["Total"] = df_covid.sum(1)
df_covid = df_covid.rename(columns={"Taarnby":"Tårnby"})

df_mean_hosp = pd.read_csv(os.path.join(path,"mean_hosp_per_mun.csv"),index_col=0)
df_mean_hosp = df_mean_hosp.rename(index={"Taarnby":"Tårnby"})

df_passenger = pd.read_csv(os.path.join(path,"passenger_decrease.csv"),index_col=0)

df_hast_daily = pd.read_csv(os.path.join(path,"speedPerMun.csv"),sep=";")
df_hast_daily = df_hast_daily.set_index("Date")

df_avg_hast_pre_lockdown = df_hast_daily[df_hast_daily.index < "2020-03-11"].mean(0)
#df_avg_hast_pre_lockdown = df_hast_daily[df_hast_daily.index >= "2020-02-15"].mean(0)


df_avg_hast_post_lockdown = df_hast_daily[df_hast_daily.index >= "2020-03-11"].mean(0)

df_hast_change = pd.DataFrame((df_avg_hast_post_lockdown - df_avg_hast_pre_lockdown)/df_avg_hast_pre_lockdown,
                             columns=["pct_increase"])

# Downloading ratio between day and night population

df_day_night = pd.read_csv(os.path.join(path,'Dag_og_natbefolk.csv'),sep=";",index_col=0)


In [3]:
def ageInt(x):
    return (int(x.split()[0]))

df_age_distribution['Age'] = df_age_distribution['Age'].apply(ageInt)

df_age_distribution = df_age_distribution[df_age_distribution['Municipality'].isin(focus_municipality)]
df_age_distribution = df_age_distribution.sort_values(['Municipality','Age'])
focus_municipality.sort()

median_ages = []
for m in focus_municipality:
    df_tmp = df_age_distribution[df_age_distribution['Municipality']== m]
    median_num = np.round(sum(df_tmp['Number'].values)/2)
    count = 0
    for a in np.unique(df_tmp['Age']):
        count += df_tmp.iloc[a]['Number']
        if count >= median_num:
            median_ages.append(a)
            break

data = {'Municipality': focus_municipality,
        'Median age': median_ages
        }

df_median_age = pd.DataFrame(data, columns = ['Municipality', 'Median age'])

df_median_age.to_csv(os.path.join(path,"Median_age.csv"))

df_popu = df_age_distribution.groupby("Municipality").sum()[["Number"]].rename(columns={"Number":"Population"})

df_popu = df_popu.loc[focus_municipality]

#Data from wikipedia
df_popu["km^2"] = [23.04,34.09,20.85,18.41,8.7,25.54,25,13.31,12.04,22.90,78.41,25.94,86.4,12.12,64.95,9.23]

df_popu["Density"] = df_popu["Population"]/df_popu["km^2"]


# Plotting of covid-19 vs. location

In [4]:
#################################################################################################################################################
#################################################################################################################################################
#################### Jitter plot to spot correlation between covid-19 and distance from center of Copenhagen. #############################
#################################################################################################################################################
#################################################################################################################################################

####################################################### set up data frame for the plot ######################################################
# coordinates for city halls
coor_copehagen = [55.675579, 12.570196]
coor_gentofte = [55.749205, 12.557447]
coor_Dragør = [55.596778, 12.637316]
coor_Frederiksberg = [55.678651, 12.532559]
coor_HøjeTaastrup = [55.652174,12.264740]
coor_Tårnby = [55.634091, 12.619329]
coor_Gladsaxe = [55.743183, 12.497172]
coor_Herlev = [55.725144, 12.432609]
coor_Ballerup = [55.734340, 12.364389]
coor_Glostrup = [55.665082, 12.394101]
coor_Hvidovre = [55.643127, 12.477458]
coor_Rødovre = [55.682674, 12.451599]
coor_Brøndby = [55.652989, 12.419191]
coor_Albertslund = [55.657088, 12.357865]
coor_Ishøj = [55.614225, 12.355672]
coor_Vallensbæk = [55.624284,12.386697]

# create a dict with coordinates
coor_town_halls = {'Albertslund':coor_Albertslund,
                  'Ballerup':coor_Ballerup,
                  'Brøndby': coor_Brøndby,
                  'Dragør': coor_Dragør,
                  'Frederiksberg': coor_Frederiksberg,
                  'Gentofte': coor_gentofte,
                  'Gladsaxe': coor_Gladsaxe,
                  'Glostrup': coor_Glostrup,
                  'Herlev': coor_Herlev,
                  'Hvidovre': coor_Hvidovre,
                  'Høje-Taastrup': coor_HøjeTaastrup,
                  'Ishøj': coor_Ishøj,
                  'København': coor_copehagen,
                  'Rødovre': coor_Rødovre,
                  'Tårnby':coor_Tårnby,
                  'Vallensbæk':coor_Vallensbæk}


# calculate the distance between copenhagen city hall and the other city halls
# list to store distances
distances = []

# radium of the earth
R = 6373.0

# coordinates for copenhagen city hall 
lat_center, lon_center = coor_town_halls['København']

# convert to radians:
lat_center = math.radians(lat_center)
lon_center = math.radians(lon_center)

for m in focus_municipality:
  # get coordinates for curent Municipality
    lat, lon = coor_town_halls[m]
  
  # convert to radians
    lat = math.radians(lat)
    lon = math.radians(lon)

  # difference i the coordinates
    dlon = lon - lon_center
    
    dlat = lat - lat_center
    
    a = math.sin(dlat / 2)**2 + math.cos(lat_center) *math.cos(lat) * math.sin(dlon / 2)**2
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    d = R*c
    
    distances.append(d)

# prepare the data frame
df_covid_dist = pd.DataFrame(index=focus_municipality)
df_covid_dist['distance'] = distances
df_covid_dist["covid"] = df_mean_hosp["mean_hosp"].to_list()
df_covid_dist = df_covid_dist[['covid','distance']]
df_covid_dist = df_covid_dist.reset_index().rename(columns={"index":"Kommune"})

#### Linear regression ###########################################################################################
X = df_covid_dist.distance.to_numpy().reshape(-1,1)
Y = df_covid_dist.covid.to_numpy().reshape(-1,1)

linear_regressor = LinearRegression()

linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)

Y_pred = list(Y_pred.squeeze())

df_covid_dist["prediction"] = Y_pred


############################################ create the plot ##################################################################################
# define source
source = ColumnDataSource(data = df_covid_dist)
# define tooltip
TOOLTIPS = [("Municipality: ", "@Kommune"),
            ("Distance", "@distance{0.1f} km"),
            ("Average hospitalisations per 100.000: ","@covid")]

# create figure
f = figure(plot_width=400,plot_height = 400,title = 'Distances to city center vs covid-19',
           x_axis_label='Distance in km',
           y_axis_label='Covid-19 hospitalisations per 100.000',      
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS)

## Apply R^2 label ##############################################################################################

cor = df_covid_dist[["distance","covid"]].corr()

R_sqrt = round(cor.iloc[0,1]**2,3)


R_sqrt_Label = Label(x=0, y=290, x_units='screen', y_units='screen',
                 text=f'  R^2 value: {R_sqrt}  ', render_mode='canvas',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0,
                 text_font_size='14pt',text_font="times")
f.add_layout(R_sqrt_Label)


## plotting #######################################################################################################

# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '20px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

# Create plot
f.circle(x = 'distance', y = 'covid', size=10,source=source)
#Create linear regression line
f.line(x="distance", y = "prediction",source=source,color="red")
# show plot
show(f)

# Covid-19 vs Median age

In [5]:
############ prepare data ##############################################################################################
covid_mean = df_mean_hosp.mean_hosp.to_list()
df_median_age_covid = df_median_age.copy()
df_median_age_covid['covid'] = covid_mean
df_median_age_covid.columns = ['Municipality','Median_age','mean_hosp']

#### Linear regression ###########################################################################################
X = df_median_age_covid.Median_age.to_numpy().reshape(-1,1)
Y = df_median_age_covid.mean_hosp.to_numpy().reshape(-1,1)

linear_regressor = LinearRegression()

linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)

Y_pred = list(Y_pred.squeeze())

df_median_age_covid["prediction"] = Y_pred


########### plot #########################################################################################################
source = ColumnDataSource(data = df_median_age_covid)

TOOLTIPS = [("Municipality: ", "@Municipality"),
            ("Median age", "@Median_age"),
            ("Average hospitalisations per 100.000: ","@mean_hosp")]
# create figure
f = figure(plot_width=400,plot_height=400,
           title = 'Median age vs covid-19 hospitalisations',
           x_axis_label='Median age',
           y_axis_label='covid 19 hospitalisations per 100.000',      
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS)

## Apply R^2 label ##############################################################################################

cor = df_median_age_covid[["Median_age","mean_hosp"]].corr()

R_sqrt = round(cor.iloc[0,1]**2,3)


R_sqrt_Label = Label(x=0, y=290, x_units='screen', y_units='screen',
                 text=f'  R^2 value: {R_sqrt}  ', render_mode='canvas',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0,
                 text_font_size='14pt',text_font="times")
f.add_layout(R_sqrt_Label)

# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '20px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

# Create plot
f.circle(x = 'Median_age', y = 'mean_hosp', size=10,source=source)
#Create linear regression line
f.line(x="Median_age", y = "prediction",source=source,color="red")
# show plot
show(f)

# Covid-19 vs. % over 65

In [6]:
############ prepare data ##############################################################################################

pct_over_65 = []

for m in focus_municipality:
    df_temp = df_age_distribution[df_age_distribution.Municipality.isin([m])]
    ages = [int(age) for age in df_temp.Age]
    df_temp["Age"] = ages
    
    pct_over_65 += [round(df_temp[df_temp.Age >= 65].Number.sum()/df_temp.Number.sum(),3)*100]
    
df_over_65 = pd.DataFrame({"Kommune":focus_municipality,
                          "pct_over_65":pct_over_65})

covid_mean = df_mean_hosp.mean_hosp.to_list()
df_over_65_covid = df_over_65.copy()
df_over_65_covid['covid'] = covid_mean
df_over_65_covid.columns = ['Kommune','Over_65','mean_hosp']

#### Linear regression ###########################################################################################
X = df_over_65_covid.Over_65.to_numpy().reshape(-1,1)
Y = df_over_65_covid.mean_hosp.to_numpy().reshape(-1,1)

linear_regressor = LinearRegression()

linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)

Y_pred = list(Y_pred.squeeze())

df_over_65_covid["prediction"] = Y_pred


######### Create figure #########################################################################################################
source = ColumnDataSource(data = df_over_65_covid)

TOOLTIPS = [("Municipality: ", "@Kommune"),
            ("Over 65 years", "@Over_65{0.2f} %"),
            ("Average hospitalisations per 100.000: ","@mean_hosp")]
# create figure
f = figure(plot_width=400,plot_height=400,
    title = 'Median age vs covid-19',
           x_axis_label='% over 65',
           y_axis_label='covid 19 hospitalisations per 100.000',      
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS)


## Apply R^2 label ##############################################################################################

cor = df_over_65_covid[["Over_65","mean_hosp"]].corr()

R_sqrt = round(cor.iloc[0,1]**2,3)


R_sqrt_Label = Label(x=0, y=290, x_units='screen', y_units='screen',
                 text=f'  R^2 value: {R_sqrt}  ', render_mode='canvas',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0,
                 text_font_size='14pt',text_font="times")
f.add_layout(R_sqrt_Label)

# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '20px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

# Create plot
f.circle(x = 'Over_65', y = 'mean_hosp', size=10,source=source)
#Create linear regression line
f.line(x="Over_65", y = "prediction",source=source,color="red")

# show plot
show(f)


/Users/tuethomsen28/Anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Covid-19 vs. Disposable income

In [7]:
############ prepare data ##############################################################################################
covid_mean = df_mean_hosp.mean_hosp.to_list()
df_income_covid = df_income.copy()
df_income_covid['covid'] = covid_mean
df_income_covid = df_income_covid.reset_index()
df_income_covid.columns = ['Kommune','Income','mean_hosp']


#### Linear regression ###########################################################################################

X = df_income_covid.Income.to_numpy().reshape(-1,1)

Y = df_income_covid.mean_hosp.to_numpy().reshape(-1,1)
#transform Y to fit linear regression 
Y = np.log(Y)

linear_regressor = LinearRegression()

linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)

#Transform Y back
Y_pred = np.exp(list(Y_pred.squeeze()))


df_income_covid["prediction"] = Y_pred


########### Create figure #########################################################################################################
source = ColumnDataSource(data = df_income_covid)

TOOLTIPS = [("Municipality: ", "@Kommune"),
            ("Disposable", "@Income"),
            ("Average hospitalisations per 100.000: ","@mean_hosp")]

# create figure
f = figure(plot_height = 400,plot_width = 400,
    title = 'Disposable Income vs covid-19',
           x_axis_label='Disposable Income',
           y_axis_label='Log covid 19 hospitalisations per 100.000',
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS,
           y_axis_type="log"
          )

## Apply R^2 label ##############################################################################################

df_income_covid["log_mean_hosp"] = np.log(df_income_covid["mean_hosp"])

cor = df_income_covid[["Income","log_mean_hosp"]].corr()

R_sqrt = round(cor.iloc[0,1]**2,3)


R_sqrt_Label = Label(x=0, y=270, x_units='screen', y_units='screen',
                 text=f'  R^2 value: {R_sqrt}  ', render_mode='canvas',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0,
                 text_font_size='14pt',text_font="times")

f.add_layout(R_sqrt_Label)

# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '20px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

# Create plot
f.circle(x = 'Income', y = 'mean_hosp', size=10,source=source)
#Create linear regression line
f.line(x="Income", y = "prediction",source=source,color="red")
# show plot
show(f)

Another method: https://kite.com/python/answers/how-to-do-exponential-and-logarithmic-curve-fitting-in-python

# Covid-19 vs. Unemployment

In [8]:
############ prepare data ##############################################################################################
df_age_distribution_working = df_age_distribution[(df_age_distribution['Age'] > 24) & (df_age_distribution['Age'] < 68)]

df_age_distribution_working = df_age_distribution_working.groupby("Municipality").sum()[["Number"]]

df_pct_unemployed = df_unemployment[["2020M02"]].T/ df_age_distribution_working.Number

covid_mean = df_mean_hosp.mean_hosp.to_list()
df_unemployment_covid = pd.DataFrame(df_pct_unemployed.T*100)
df_unemployment_covid['covid'] = covid_mean
df_unemployment_covid = df_unemployment_covid.reset_index()
df_unemployment_covid.columns = ['Kommune','Unemployment','mean_hosp']


#### Linear regression ###########################################################################################
X = df_unemployment_covid.Unemployment.to_numpy().reshape(-1,1)
Y = df_unemployment_covid.mean_hosp.to_numpy().reshape(-1,1)

linear_regressor = LinearRegression()

linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)

Y_pred = list(Y_pred.squeeze())

df_unemployment_covid["predicted_hosp"] = Y_pred


########### Create figure #########################################################################################################
source = ColumnDataSource(data = df_unemployment_covid)

TOOLTIPS = [("Municipality: ", "@Kommune"),
            ("Unemployment rate", "@Unemployment{0.2f}%"),
            ("Average hospitalisations per 100.000: ","@mean_hosp")]

# create figure
f = figure(plot_height = 400, plot_width = 400,
    title = '% unemployment vs covid-19',
           x_axis_label='Unemployment rate (%)',
           y_axis_label='Covid 19 hospitalisations per 100.000',
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS,
           #y_axis_type="log"
          )


## Apply R^2 label ##############################################################################################

cor = df_unemployment_covid[["Unemployment","mean_hosp"]].corr()

R_sqrt = round(cor.iloc[0,1]**2,3)


R_sqrt_Label = Label(x=0, y=290, x_units='screen', y_units='screen',
                 text=f'  R^2 value: {R_sqrt}  ', render_mode='canvas',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0,
                 text_font_size='14pt',text_font="times")
f.add_layout(R_sqrt_Label)

## Plot ####################################################################################################

# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '20px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

# Create plot
f.circle(x = 'Unemployment', y = 'mean_hosp', size=10,source=source)

#Create linear regression line
f.line(x="Unemployment", y = "predicted_hosp",source=source,color="red")

# show plot
show(f)


# Covid-19 vs. Speed increase

In [9]:
############ prepare data ##############################################################################################

df_temp = df_hast_change.copy()

df_covid_max = df_covid[df_covid.index < "2020-04-01"].max()

df_covid_avg_after = df_covid[df_covid.index > "2020-04-01"].mean(0)

df_covid_decrease = pd.DataFrame((df_covid_max - df_covid_avg_after) / df_covid_max,columns=["covid_decrease"])

df_covid_decrease["speed_increase"] = df_temp["pct_increase"]

df_covid_decrease = df_covid_decrease.iloc[:-1]

df_covid_decrease = df_covid_decrease.reset_index()

df_covid_decrease.columns = ['Kommune','Covid_decrease','Speed_increase']

#df_covid_decrease = df_covid_decrease[df_covid_decrease.Kommune != "Ishøj"]

#### Linear regression ###########################################################################################
X = df_covid_decrease.Speed_increase.to_numpy().reshape(-1,1)
Y = df_covid_decrease.Covid_decrease.to_numpy().reshape(-1,1)

linear_regressor = LinearRegression()

linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)

Y_pred = list(Y_pred.squeeze())

df_covid_decrease["predicted_decrease"] = Y_pred


########### Create figure #########################################################################################################
source = ColumnDataSource(data = df_covid_decrease)

TOOLTIPS = [("Municipality: ", "@Kommune"),
            ("Increase in speed", "@Speed_increase{%0.2f}"),
            ("Decrease in hospilitations: ","@Covid_decrease{%0.2f}")]

# create figure
f = figure(plot_width=400,plot_height=400,
    title = '% increase in speed vs. covid-19',
           x_axis_label='Increase in speed (%)',
           y_axis_label='Decrease in hospitalisations (%)',
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS,
           #y_axis_type="log"
          )

## Apply R^2 label ##############################################################################################

cor = df_covid_decrease[["Speed_increase","Covid_decrease"]].corr()

R_sqrt = round(cor.iloc[0,1]**2,3)


R_sqrt_Label = Label(x=0, y=300, x_units='screen', y_units='screen',
                 text=f'  R^2 value: {R_sqrt}  ', render_mode='canvas',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0,
                 text_font_size='14pt',text_font="times")
f.add_layout(R_sqrt_Label)


## Create style and plot circles and line ###################################################################################

# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '16px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

# Create circle
f.circle(x = 'Speed_increase', y = 'Covid_decrease', size=10,source=source)

#Create linear regression line
f.line(x="Speed_increase", y = "predicted_decrease",source=source,color="red")

# show plot
show(f)

# Covid-19 vs. education level

In [10]:
############ prepare data ##############################################################################################
covid_mean = df_mean_hosp.mean_hosp.to_list()
df_education_covid = pd.DataFrame(df_pct_high_education)
df_education_covid['covid'] = covid_mean
df_education_covid = df_education_covid.reset_index()
df_education_covid.columns = ['Kommune','pct_highly_educated','mean_hosp']


#### Linear regression ###########################################################################################
X = df_education_covid.pct_highly_educated.to_numpy().reshape(-1,1)
Y = df_education_covid.mean_hosp.to_numpy().reshape(-1,1)

linear_regressor = LinearRegression()

linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)

Y_pred = list(Y_pred.squeeze())

df_education_covid["predicted_hosp"] = Y_pred


########### Create figure #########################################################################################################
source = ColumnDataSource(data = df_education_covid)

TOOLTIPS = [("Municipality: ", "@Kommune"),
            ("% highly educated", "@pct_highly_educated{%0.2f}"),
            ("Average hospitalisations per 100.000: ","@mean_hosp")]

# create figure
f = figure(plot_height = 400, plot_width = 400,
    title = 'High education vs covid-19',
           x_axis_label='% highly educated',
           y_axis_label='Covid 19 hospitalisations per 100.000',
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS,
           #y_axis_type="log"
          )


## Apply R^2 label ##############################################################################################

cor = df_education_covid[["pct_highly_educated","mean_hosp"]].corr()

R_sqrt = round(cor.iloc[0,1]**2,3)


R_sqrt_Label = Label(x=0, y=290, x_units='screen', y_units='screen',
                 text=f'  R^2 value: {R_sqrt}  ', render_mode='canvas',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0,
                 text_font_size='14pt',text_font="times")
f.add_layout(R_sqrt_Label)

## Plot ####################################################################################################

# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '20px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

# Create plot
f.circle(x = 'pct_highly_educated', y = 'mean_hosp', size=10,source=source)

#Create linear regression line
f.line(x="pct_highly_educated", y = "predicted_hosp",source=source,color="red")

# show plot
show(f)

# Covid-19 vs. passenger decrease

In [11]:
############ prepare data ##############################################################################################
df_covid_passenger = df_passenger/100

df_temp = df_covid_decrease.copy().set_index("Kommune")["Covid_decrease"]

df_covid_passenger["covid_decrease"] = df_temp

df_covid_passenger = df_covid_passenger.reset_index()

#### Linear regression ###########################################################################################
X = df_covid_passenger.pct_drop_2020.to_numpy().reshape(-1,1)
Y = df_covid_passenger.covid_decrease.to_numpy().reshape(-1,1)

linear_regressor = LinearRegression()

linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)

Y_pred = list(Y_pred.squeeze())

df_covid_passenger["predicted_decrease"] = Y_pred



########### Create figure #########################################################################################################
source = ColumnDataSource(data = df_covid_passenger)

TOOLTIPS = [("Municipality: ", "@Kommune"),
            ("Decrease in passengers", "@pct_drop_2020{%0.2f}"),
            ("Decrease in hospilitations: ","@covid_decrease{%0.2f}")]

# create figure
f = figure(plot_width=400,plot_height=400,
    title = '% decrease in passengers vs. covid-19',
           x_axis_label='Decrease in passengers (%)',
           y_axis_label='Decrease in hospitalisations (%)',
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS,
           #y_axis_type="log"
          )

## Apply R^2 label ##############################################################################################

cor = df_covid_passenger[["per_2020","covid_decrease"]].corr()

R_sqrt = round(cor.iloc[0,1]**2,3)


R_sqrt_Label = Label(x=50, y=290, x_units='screen', y_units='screen',
                 text=f'  R^2 value: {R_sqrt}  ', render_mode='canvas',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0,
                 text_font_size='14pt',text_font="times")

f.add_layout(R_sqrt_Label)

## plot #########################################################################################################

# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '20px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

# Create plot
f.circle(x = 'pct_drop_2020', y = 'covid_decrease', size=10,source=source)

#Create linear regression line
f.line(x="pct_drop_2020", y = "predicted_decrease",source=source,color="red")

# show plot
show(f)


# Covid-19 vs. Ethnicity

In [12]:
############ prepare data ##############################################################################################
df_covid_ethnicity = df_pct_ethnicity.copy()

df_temp = df_covid_decrease.copy().set_index("Kommune")["Covid_decrease"]

df_covid_ethnicity["covid_decrease"] = df_temp

df_covid_ethnicity = df_covid_ethnicity.reset_index()

df_covid_ethnicity.columns = ["Kommune","pct_not_vestern","covid_decrease"]

#### Linear regression ###########################################################################################
X = df_covid_ethnicity.pct_not_vestern.to_numpy().reshape(-1,1)
Y = df_covid_ethnicity.covid_decrease.to_numpy().reshape(-1,1)

linear_regressor = LinearRegression()

linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)

Y_pred = list(Y_pred.squeeze())

df_covid_ethnicity["predicted_decrease"] = Y_pred



########### Create figure #########################################################################################################
source = ColumnDataSource(data = df_covid_ethnicity)

TOOLTIPS = [("Municipality: ", "@Kommune"),
            ("% non-vestern", "@pct_not_vestern{%0.2f}"),
            ("Decrease in hospilitations: ","@covid_decrease{%0.2f}")]

# create figure
f = figure(plot_width=400,plot_height=400,
    title = '% non-vestern vs. Covid-19',
           x_axis_label='Non-vestern(%)',
           y_axis_label='Decrease in hospitalisations (%)',
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS,
           #y_axis_type="log"
          )

## Apply R^2 label ##############################################################################################

cor = df_covid_ethnicity[["pct_not_vestern","covid_decrease"]].corr()

R_sqrt = round(cor.iloc[0,1]**2,3)


R_sqrt_Label = Label(x=50, y=290, x_units='screen', y_units='screen',
                 text=f'  R^2 value: {R_sqrt}  ', render_mode='canvas',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0,
                 text_font_size='14pt',text_font="times")

f.add_layout(R_sqrt_Label)

## plot #########################################################################################################

# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '20px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

# Create plot
f.circle(x = 'pct_not_vestern', y = 'covid_decrease', size=10,source=source)

#Create linear regression line
f.line(x="pct_not_vestern", y = "predicted_decrease",source=source,color="red")


# show plot
show(f)

# Passenger decrease vs. Density

In [13]:
############ prepare data ##############################################################################################
df_passenger_density = df_popu[["Density"]].copy()
df_temp = df_passenger.copy()/100
df_passenger_density["pct_drop_2020"] = df_temp["pct_drop_2020"]
df_passenger_density = df_passenger_density.reset_index()
df_passenger_density.columns = ["Kommune","Density","pct_drop_2020"]


#df_passenger_ethnicity = df_passenger_ethnicity[df_passenger_ethnicity.Kommune != "Albertslund"]

#### Linear regression ###########################################################################################
X = df_passenger_density.Density.to_numpy().reshape(-1,1)
Y = df_passenger_density.pct_drop_2020.to_numpy().reshape(-1,1)

linear_regressor = LinearRegression()

linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)

Y_pred = list(Y_pred.squeeze())

df_passenger_density["predicted_decrease"] = Y_pred



########### Create figure #########################################################################################################
source = ColumnDataSource(data = df_passenger_density)

TOOLTIPS = [("Municipality: ", "@Kommune"),
            ("People per square-kilometer", "@Density km^2"),
            ("% decrease in passengers ","@pct_drop_2020{%0.2f}")]

# create figure
f = figure(plot_width=400,plot_height=400,
    title = 'Passenger decrease vs. Density',
           x_axis_label='People per square-kilometer ',
           y_axis_label='Decrease in hospitalisations (%)',
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS,
           #y_axis_type="log"
          )

## Apply R^2 label ##############################################################################################

cor = df_passenger_density[["Density","pct_drop_2020"]].corr()

R_sqrt = round(cor.iloc[0,1]**2,3)


R_sqrt_Label = Label(x=0, y=290, x_units='screen', y_units='screen',
                 text=f'  R^2 value: {R_sqrt}  ', render_mode='canvas',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0,
                 text_font_size='14pt',text_font="times")

f.add_layout(R_sqrt_Label)

## plot #########################################################################################################

# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '20px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

# Create plot
f.circle(x = 'Density', y = 'pct_drop_2020', size=10,source=source)

#Create linear regression line
f.line(x="Density", y = "predicted_decrease",source=source,color="red")


# show plot
show(f)

# Passenger decrease vs. Day/Night Ratio

In [14]:
############ prepare data ##############################################################################################
df_passenger_DayNightRatio = df_day_night[["NatDagRatio"]].copy()
df_temp = df_passenger.copy()/100
df_passenger_DayNightRatio["pct_drop_2020"] = df_temp["pct_drop_2020"]
df_passenger_DayNightRatio = df_passenger_DayNightRatio.reset_index()
df_passenger_DayNightRatio.columns = ["Kommune","DayNightRatio","pct_drop_2020"]

#df_passenger_ethnicity = df_passenger_ethnicity[df_passenger_ethnicity.Kommune != "Albertslund"]

#### Linear regression ###########################################################################################
X = df_passenger_DayNightRatio.DayNightRatio.to_numpy().reshape(-1,1)
Y = df_passenger_DayNightRatio.pct_drop_2020.to_numpy().reshape(-1,1)

linear_regressor = LinearRegression()

linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)

Y_pred = list(Y_pred.squeeze())

df_passenger_DayNightRatio["predicted_decrease"] = Y_pred



########### Create figure #########################################################################################################
source = ColumnDataSource(data = df_passenger_DayNightRatio)

TOOLTIPS = [("Municipality: ", "@Kommune"),
            ("Day Night Ratio", "@DayNightRatio"),
            ("% decrease in passengers ","@pct_drop_2020{%0.2f}")]

# create figure
f = figure(plot_width=400,plot_height=400,
    title = 'Decrease in passengers vs. Day/Night Ratio',
           x_axis_label='People per square-kilometer ',
           y_axis_label='Decrease in hospitalisations (%)',
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS,
           #y_axis_type="log"
          )

## Apply R^2 label ##############################################################################################

cor = df_passenger_DayNightRatio[["DayNightRatio","pct_drop_2020"]].corr()

R_sqrt = round(cor.iloc[0,1]**2,3)


R_sqrt_Label = Label(x=0, y=290, x_units='screen', y_units='screen',
                 text=f'  R^2 value: {R_sqrt}  ', render_mode='canvas',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0,
                 text_font_size='14pt',text_font="times")

f.add_layout(R_sqrt_Label)

## plot #########################################################################################################

# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '20px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

# Create plot
f.circle(x = 'DayNightRatio', y = 'pct_drop_2020', size=10,source=source)

#Create linear regression line
f.line(x="DayNightRatio", y = "predicted_decrease",source=source,color="red")


# show plot
show(f)

# Passenger decrease vs. non-vestern

In [15]:
############ prepare data ##############################################################################################
df_passenger_ethnicity = df_pct_ethnicity.copy()


df_temp = df_passenger.copy()/100

df_passenger_ethnicity["pct_drop_2020"] = df_temp[["pct_drop_2020"]]

df_passenger_ethnicity = df_passenger_ethnicity.reset_index()

df_passenger_ethnicity.columns = ["Kommune","pct_not_vestern","pct_drop_2020"]

#df_passenger_ethnicity = df_passenger_ethnicity[df_passenger_ethnicity.Kommune != "Albertslund"]

#### Linear regression ###########################################################################################
X = df_passenger_ethnicity.pct_not_vestern.to_numpy().reshape(-1,1)
Y = df_passenger_ethnicity.pct_drop_2020.to_numpy().reshape(-1,1)

linear_regressor = LinearRegression()

linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)

Y_pred = list(Y_pred.squeeze())

df_passenger_ethnicity["predicted_decrease"] = Y_pred



########### Create figure #########################################################################################################
source = ColumnDataSource(data = df_passenger_ethnicity)

TOOLTIPS = [("Municipality: ", "@Kommune"),
            ("% non-vestern", "@pct_not_vestern{%0.2f}"),
            ("% decrease in passengers ","@pct_drop_2020{%0.2f}")]

# create figure
f = figure(plot_width=400,plot_height=400,
    title = '% non-vestern vs. decrease in passengers',
           x_axis_label='Non-vestern(%)',
           y_axis_label='Decrease in hospitalisations (%)',
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS,
           #y_axis_type="log"
          )

## Apply R^2 label ##############################################################################################

cor = df_passenger_ethnicity[["pct_not_vestern","pct_drop_2020"]].corr()

R_sqrt = round(cor.iloc[0,1]**2,3)


R_sqrt_Label = Label(x=0, y=290, x_units='screen', y_units='screen',
                 text=f'  R^2 value: {R_sqrt}  ', render_mode='canvas',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0,
                 text_font_size='14pt',text_font="times")

f.add_layout(R_sqrt_Label)

## plot #########################################################################################################

# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '20px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

# Create plot
f.circle(x = 'pct_not_vestern', y = 'pct_drop_2020', size=10,source=source)

#Create linear regression line
f.line(x="pct_not_vestern", y = "predicted_decrease",source=source,color="red")


# show plot
show(f)

# Passenger decrease vs. unemployment

In [16]:
############ prepare data ##############################################################################################
df_age_distribution_working = df_age_distribution[(df_age_distribution['Age'] > 24) & (df_age_distribution['Age'] < 68)]

df_age_distribution_working = df_age_distribution_working.groupby("Municipality").sum()[["Number"]]

df_pct_unemployed = df_unemployment[["2020M02"]].T / df_age_distribution_working.Number

df_temp = df_pct_unemployed.copy().T.rename(columns={"2020M02":"pct_unemployed"})

df_unemployement_passenger = df_passenger/100

df_unemployement_passenger["pct_unemployed"] = df_temp["pct_unemployed"]

#df_unemployement_passenger = df_unemployement_passenger[df_unemployement_passenger.index != "Vallensbæk"]

#### Linear regression ###########################################################################################
X = df_unemployement_passenger.pct_unemployed.to_numpy().reshape(-1,1)
Y = df_unemployement_passenger.pct_drop_2020.to_numpy().reshape(-1,1)

linear_regressor = LinearRegression()

linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)

Y_pred = list(Y_pred.squeeze())

df_unemployement_passenger["prediction"] = Y_pred



########### Create figure #########################################################################################################
source = ColumnDataSource(data = df_unemployement_passenger)

TOOLTIPS = [("Municipality: ", "@Kommune"),
            ("Unemployed", "@pct_unemployed{%0.2f}"),
            ("Decrease in passengers: ","@pct_drop_2020{%0.2f}")]

# create figure
f = figure(plot_height = 400,plot_width=400,
    title = '% unemployed vs. decrease in passengers',
           x_axis_label='% unemployed',
           y_axis_label='% decrease in passengers',
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS,
           #y_axis_type="log"
          )

## Apply R^2 label ##############################################################################################

cor = df_unemployement_passenger[["pct_drop_2020","pct_unemployed"]].corr()

R_sqrt = round(cor.iloc[0,1]**2,3)


R_sqrt_Label = Label(x=0, y=290, x_units='screen', y_units='screen',
                 text=f'  R^2 value: {R_sqrt}  ', render_mode='canvas',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0,
                 text_font_size='14pt',text_font="times")

f.add_layout(R_sqrt_Label)

## plot #########################################################################################################

# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '20px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

# Create plot
f.circle(x = 'pct_unemployed', y = 'pct_drop_2020', size=10,source=source)

#Create linear regression line
f.line(x="pct_unemployed", y = "prediction",source=source,color="red")


# show plot
show(f)

# Passenger decrease vs. income

In [17]:
############ prepare data ##############################################################################################
df_temp = df_income.copy()

df_income_passenger = df_passenger/100

df_income_passenger["Income"] = df_temp["Income"]

#df_income_passenger = df_income_passenger[df_income_passenger.index != "Gentofte"]

#### Linear regression ###########################################################################################
X = df_income_passenger.Income.to_numpy().reshape(-1,1)
Y = df_income_passenger.pct_drop_2020.to_numpy().reshape(-1,1)

linear_regressor = LinearRegression()

linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)

Y_pred = list(Y_pred.squeeze())

df_income_passenger["prediction"] = Y_pred

########### Create figure #########################################################################################################
source = ColumnDataSource(data = df_income_passenger)

TOOLTIPS = [("Municipality: ", "@Kommune"),
            ("Income", "@Income kr."),
            ("Decrease in passengers: ","@pct_drop_2020{%0.2f}")]

# create figure
f = figure(plot_width=400,plot_height=400,
    title = 'Disposable income vs. decrease in passengers',
           x_axis_label='Income',
           y_axis_label='% decrease in passengers',
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS,
           y_axis_type="log"
           #x_axis_type="log"
          )

## Apply R^2 label ##############################################################################################

cor = df_income_passenger[["pct_drop_2020","Income"]].corr()

R_sqrt = round(cor.iloc[0,1]**2,3)


R_sqrt_Label = Label(x=50, y=290, x_units='screen', y_units='screen',
                 text=f'  R^2 value: {R_sqrt}  ', render_mode='canvas',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0,
                 text_font_size='14pt',text_font="times")

f.add_layout(R_sqrt_Label)

## plot #########################################################################################################

# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '20px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

# Create plot
f.circle(x = 'Income', y = 'pct_drop_2020', size=10,source=source)

#Create linear regression line
f.line(x="Income", y = "prediction",source=source,color="red")

# show plot
show(f)

# Passenger decrease vs. education level

In [18]:
############ prepare data ##############################################################################################

df_temp = df_pct_high_education

df_education_passenger = df_passenger/100

df_education_passenger["pct_highly_educated"] = df_temp["pct_highly_educated"]

#df_education_passenger = df_education_passenger[df_education_passenger.index != "Albertslund"]

#### Linear regression ###########################################################################################
X = df_education_passenger.pct_highly_educated.to_numpy().reshape(-1,1)
Y = df_education_passenger.pct_drop_2020.to_numpy().reshape(-1,1)

linear_regressor = LinearRegression()

linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)

Y_pred = list(Y_pred.squeeze())

df_education_passenger["prediction"] = Y_pred



########### Create figure #########################################################################################################
source = ColumnDataSource(data = df_education_passenger)

TOOLTIPS = [("Municipality: ", "@Kommune"),
            ("High education", "@pct_highly_educated{%0.2f}"),
            ("Decrease in passengers: ","@pct_drop_2020{%0.2f}")]

# create figure
f = figure(plot_height = 400,plot_width=400,
    title = '% highly educated vs. decrease in passengers',
           x_axis_label='% high education',
           y_axis_label='% decrease in passengers',
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS,
           #y_axis_type="log"
          )

## Apply R^2 label ##############################################################################################

cor = df_education_passenger[["pct_drop_2020","pct_highly_educated"]].corr()

R_sqrt = round(cor.iloc[0,1]**2,3)


R_sqrt_Label = Label(x=150, y=290, x_units='screen', y_units='screen',
                 text=f'  R^2 value: {R_sqrt}  ', render_mode='canvas',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0,
                 text_font_size='14pt',text_font="times")

f.add_layout(R_sqrt_Label)

## plot #########################################################################################################

# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '20px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

# Create plot
f.circle(x = 'pct_highly_educated', y = 'pct_drop_2020', size=10,source=source)

#Create linear regression line
f.line(x="pct_highly_educated", y = "prediction",source=source,color="red")


# show plot
show(f)

# Passenger decrease vs. median age

In [19]:
############ prepare data ##############################################################################################

df_temp = df_median_age.copy().set_index("Municipality")

df_age_passenger = df_passenger/100

df_age_passenger["Median_age"] = df_temp["Median age"]

#df_age_passenger = df_age_passenger[df_age_passenger.index != "Vallensbæk"]

#### Linear regression ###########################################################################################
X = df_age_passenger.Median_age.to_numpy().reshape(-1,1)
Y = df_age_passenger.pct_drop_2020.to_numpy().reshape(-1,1)

linear_regressor = LinearRegression()

linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)

Y_pred = list(Y_pred.squeeze())

df_age_passenger["prediction"] = Y_pred



########### Create figure #########################################################################################################
source = ColumnDataSource(data = df_age_passenger)

TOOLTIPS = [("Municipality: ", "@Kommune"),
            ("Median age", "@Median_age years"),
            ("Decrease in passengers: ","@per_2020{%0.2f}")]

# create figure
f = figure(plot_width=400,plot_height=400,
    title = 'Median age vs. decrease in passengers',
           x_axis_label='Median age',
           y_axis_label='Rate of decrease in passengers',
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS,
           #y_axis_type="log"
          )

## Apply R^2 label ##############################################################################################

cor = df_age_passenger[["pct_drop_2020","Median_age"]].corr()

R_sqrt = round(cor.iloc[0,1]**2,3)


R_sqrt_Label = Label(x=150, y=290, x_units='screen', y_units='screen',
                 text=f'  R^2 value: {R_sqrt}  ', render_mode='canvas',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0,
                 text_font_size='14pt',text_font="times")

f.add_layout(R_sqrt_Label)

## plot #########################################################################################################

# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '20px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

# Create plot
f.circle(x = 'Median_age', y = 'pct_drop_2020', size=10,source=source)

#Create linear regression line
f.line(x="Median_age", y = "prediction",source=source,color="red")


# show plot
show(f)

# Speed vs. Day/Night ratio

In [20]:
#################################################################################################################################################
#################################################################################################################################################
############################################################ Median age vs. avg. increase in speed ##############################################
#################################################################################################################################################
#################################################################################################################################################

############ prepare data ##############################################################################################

df_daynight_age_avg_speed = df_day_night[["NatDagRatio"]].copy()

df_daynight_age_avg_speed["speed_change"] = df_hast_change

df_daynight_age_avg_speed = df_daynight_age_avg_speed.reset_index()
df_daynight_age_avg_speed.columns = ['Municipality','DayNightRatio','Speed_change']

#### Linear regression ###########################################################################################
X = df_daynight_age_avg_speed.DayNightRatio.to_numpy().reshape(-1,1)
Y = df_daynight_age_avg_speed.Speed_change.to_numpy().reshape(-1,1)

linear_regressor = LinearRegression()

linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)

Y_pred = list(Y_pred.squeeze())

df_daynight_age_avg_speed["prediction"] = Y_pred


########### plot #########################################################################################################
source = ColumnDataSource(data = df_daynight_age_avg_speed)

TOOLTIPS = [("Municipality: ", "@Municipality"),
            ("Median age", "@Median_age"),
            ("Speed change","@Speed_change{%0.2f}")]
# create figure
f = figure(plot_width=400,plot_height=400,
           title = 'Day/Night Ratio vs. speed increase',
           x_axis_label='Day/Night Ratio',
           y_axis_label='Increase in speed (%)',      
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS)


## Apply R^2 label ##############################################################################################

cor = df_daynight_age_avg_speed[["Speed_change","DayNightRatio"]].corr()

R_sqrt = round(cor.iloc[0,1]**2,3)


R_sqrt_Label = Label(x=175, y=295, x_units='screen', y_units='screen',
                 text=f'  R^2 value: {R_sqrt}  ', render_mode='canvas',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0,
                 text_font_size='14pt',text_font="times")

f.add_layout(R_sqrt_Label)

## Plot figures ################################################################################################

# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '20px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

# Create plot
f.circle(x = 'DayNightRatio', y = 'Speed_change', size=10,source=source)

#Create linear regression line
f.line(x="DayNightRatio", y = "prediction",source=source,color="red")


# show plot
show(f)

# Speed vs. Density

In [21]:
#################################################################################################################################################
#################################################################################################################################################
############################################################ Median age vs. avg. increase in speed ##############################################
#################################################################################################################################################
#################################################################################################################################################

############ prepare data ##############################################################################################

df_density_age_avg_speed = df_popu[["Density"]].copy()

df_density_age_avg_speed["speed_change"] = df_hast_change

df_density_age_avg_speed = df_density_age_avg_speed.reset_index()
df_density_age_avg_speed.columns = ['Municipality','Density','Speed_change']

#### Linear regression ###########################################################################################
X = df_density_age_avg_speed.Density.to_numpy().reshape(-1,1)
Y = df_density_age_avg_speed.Speed_change.to_numpy().reshape(-1,1)

linear_regressor = LinearRegression()

linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)

Y_pred = list(Y_pred.squeeze())

df_density_age_avg_speed["prediction"] = Y_pred


########### plot #########################################################################################################
source = ColumnDataSource(data = df_density_age_avg_speed)

TOOLTIPS = [("Municipality: ", "@Municipality"),
            ("People per square-kilometer", "@Density km^2"),
            ("Speed change","@Speed_change{%0.2f}")]
# create figure
f = figure(plot_width=400,plot_height=400,
           title = 'Day/Night Ratio vs. speed increase',
           x_axis_label='Day/Night Ratio',
           y_axis_label='Increase in speed (%)',      
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS,
           #x_axis_type="log"
          )


## Apply R^2 label ##############################################################################################

cor = df_density_age_avg_speed[["Speed_change","Density"]].corr()

R_sqrt = round(cor.iloc[0,1]**2,3)


R_sqrt_Label = Label(x=175, y=295, x_units='screen', y_units='screen',
                 text=f'  R^2 value: {R_sqrt}  ', render_mode='canvas',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0,
                 text_font_size='14pt',text_font="times")

f.add_layout(R_sqrt_Label)

## Plot figures ################################################################################################

# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '20px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

# Create plot
f.circle(x = 'Density', y = 'Speed_change', size=10,source=source)

#Create linear regression line
f.line(x="Density", y = "prediction",source=source,color="red")

# show plot
show(f)

# Speed vs. Median age

In [22]:
#################################################################################################################################################
#################################################################################################################################################
############################################################ Median age vs. avg. increase in speed ##############################################
#################################################################################################################################################
#################################################################################################################################################

############ prepare data ##############################################################################################

df_median_age_avg_speed = df_median_age.copy()

df_median_age_avg_speed = df_median_age_avg_speed.set_index("Municipality")

df_median_age_avg_speed["speed_change"] = df_hast_change

df_median_age_avg_speed = df_median_age_avg_speed.reset_index()
df_median_age_avg_speed.columns = ['Municipality','Median_age','Speed_change']

#### Linear regression ###########################################################################################
X = df_median_age_avg_speed.Median_age.to_numpy().reshape(-1,1)
Y = df_median_age_avg_speed.Speed_change.to_numpy().reshape(-1,1)

linear_regressor = LinearRegression()

linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)

Y_pred = list(Y_pred.squeeze())

df_median_age_avg_speed["prediction"] = Y_pred


########### plot #########################################################################################################
source = ColumnDataSource(data = df_median_age_avg_speed)

TOOLTIPS = [("Municipality: ", "@Municipality"),
            ("Median age", "@Median_age"),
            ("Speed change","@Speed_change{%0.2f}")]
# create figure
f = figure(plot_width=400,plot_height=400,
           title = 'Median age vs change in avg. speed',
           x_axis_label='Median age',
           y_axis_label='Increase in speed (%)',      
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS)


## Apply R^2 label ##############################################################################################

cor = df_median_age_avg_speed[["Speed_change","Median_age"]].corr()

R_sqrt = round(cor.iloc[0,1]**2,3)


R_sqrt_Label = Label(x=175, y=295, x_units='screen', y_units='screen',
                 text=f'  R^2 value: {R_sqrt}  ', render_mode='canvas',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0,
                 text_font_size='14pt',text_font="times")

f.add_layout(R_sqrt_Label)

## Plot figures ################################################################################################

# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '20px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

# Create plot
f.circle(x = 'Median_age', y = 'Speed_change', size=10,source=source)

#Create linear regression line
f.line(x="Median_age", y = "prediction",source=source,color="red")


# show plot
show(f)

# Speed vs. Education Level

In [23]:
############ prepare data ##############################################################################################

df_education_speed = df_hast_change.copy()

df_education_speed["pct_highly_educated"] = df_pct_high_education["pct_highly_educated"]

df_education_speed = df_education_speed.reset_index()

df_education_speed.columns = ['Kommune','Speed_increase','pct_highly_educated']

#df_covid_decrease = df_covid_decrease[df_covid_decrease.Kommune != "Ishøj"]

#### Linear regression ###########################################################################################
X = df_education_speed.pct_highly_educated.to_numpy().reshape(-1,1)
Y = df_education_speed.Speed_increase.to_numpy().reshape(-1,1)

linear_regressor = LinearRegression()

linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)

Y_pred = list(Y_pred.squeeze())

df_education_speed["prediction"] = Y_pred


########### Create figure #########################################################################################################
source = ColumnDataSource(data = df_education_speed)

TOOLTIPS = [("Municipality: ", "@Kommune"),
            ("Increase in speed", "@Speed_increase{%0.2f}"),
            ("Highly educated (%): ","@pct_highly_educated{%0.2f}")]

# create figure
f = figure(plot_width=400,plot_height=400,
    title = '% increase in speed vs. Education level',
           x_axis_label='Highly educated (%)',
           y_axis_label='Increase in speed (%)',
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS,
           #y_axis_type="log"
          )

## Apply R^2 label ##############################################################################################

cor = df_education_speed[["Speed_increase","pct_highly_educated"]].corr()

R_sqrt = round(cor.iloc[0,1]**2,3)


R_sqrt_Label = Label(x=0, y=300, x_units='screen', y_units='screen',
                 text=f'  R^2 value: {R_sqrt}  ', render_mode='canvas',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0,
                 text_font_size='14pt',text_font="times")
f.add_layout(R_sqrt_Label)


## Create style and plot circles and line ###################################################################################

# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '16px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

# Create circle
f.circle(x = 'pct_highly_educated', y = 'Speed_increase', size=10,source=source)

#Create linear regression line
f.line(x="pct_highly_educated", y = "prediction",source=source,color="red")

# show plot
show(f)

# Speed vs. unemployment

In [24]:
############ prepare data ##############################################################################################

df_age_distribution_working = df_age_distribution[(df_age_distribution['Age'] > 24) & (df_age_distribution['Age'] < 68)]

df_age_distribution_working = df_age_distribution_working.groupby("Municipality").sum()[["Number"]]

df_pct_unemployed = df_unemployment[["2020M02"]].T / df_age_distribution_working.Number

df_temp = df_pct_unemployed.copy().T.rename(columns={"2020M02":"pct_unemployed"})

df_unemployement_speed = df_hast_change.copy()

df_unemployement_speed["pct_unemployed"] = df_temp["pct_unemployed"]

df_unemployement_speed = df_unemployement_speed.reset_index()

#df_unemployement_passenger = df_unemployement_passenger[df_unemployement_passenger.index != "Vallensbæk"]

#### Linear regression ###########################################################################################
X = df_unemployement_speed.pct_unemployed.to_numpy().reshape(-1,1)
Y = df_unemployement_speed.pct_increase.to_numpy().reshape(-1,1)

linear_regressor = LinearRegression()

linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)

Y_pred = list(Y_pred.squeeze())

df_unemployement_speed["prediction"] = Y_pred



########### Create figure #########################################################################################################
source = ColumnDataSource(data = df_unemployement_speed)

TOOLTIPS = [("Municipality: ", "@index"),
            ("Unemployed", "@pct_unemployed{%0.2f}"),
            ("Increase in speed (%): ","@pct_increase{%0.2f}")]

# create figure
f = figure(plot_height = 400,plot_width=400,
    title = '% unemployed vs. decrease in passengers',
           x_axis_label='% unemployed',
           y_axis_label='% increase in speed',
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS,
           #y_axis_type="log"
          )

## Apply R^2 label ##############################################################################################

cor = df_unemployement_speed[["pct_increase","pct_unemployed"]].corr()

R_sqrt = round(cor.iloc[0,1]**2,3)


R_sqrt_Label = Label(x=0, y=290, x_units='screen', y_units='screen',
                 text=f'  R^2 value: {R_sqrt}  ', render_mode='canvas',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0,
                 text_font_size='14pt',text_font="times")

f.add_layout(R_sqrt_Label)

## plot #########################################################################################################

# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '20px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

# Create plot
f.circle(x = 'pct_unemployed', y = 'pct_increase', size=10,source=source)

#Create linear regression line
f.line(x="pct_unemployed", y = "prediction",source=source,color="red")


# show plot
show(f)

# Speed vs. income

In [25]:
############ prepare data ##############################################################################################
df_temp = df_income.copy()

df_income_speed = df_hast_change.copy()

df_income_speed["Income"] = df_temp["Income"]

#### Linear regression ###########################################################################################
X = df_income_speed.Income.to_numpy().reshape(-1,1)
Y = df_income_speed.pct_increase.to_numpy().reshape(-1,1)

linear_regressor = LinearRegression()

linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)

Y_pred = list(Y_pred.squeeze())

df_income_speed["prediction"] = Y_pred



########### Create figure #########################################################################################################
source = ColumnDataSource(data = df_income_speed)

TOOLTIPS = [("Municipality: ", "@index"),
            ("Income", "@Income{%0.2f}"),
            ("Decrease in passengers: ","@pct_increase{%0.2f}")]

# create figure
f = figure(plot_width=400,plot_height=400,
    title = 'Disposable income vs. increase in speed',
           x_axis_label='Income',
           y_axis_label='% increase in speed',
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS,
           #y_axis_type="log"
          )

## Apply R^2 label ##############################################################################################

cor = df_income_speed[["pct_increase","Income"]].corr()

R_sqrt = round(cor.iloc[0,1]**2,3)


R_sqrt_Label = Label(x=150, y=290, x_units='screen', y_units='screen',
                 text=f'  R^2 value: {R_sqrt}  ', render_mode='canvas',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0,
                 text_font_size='14pt',text_font="times")

f.add_layout(R_sqrt_Label)

## plot #########################################################################################################

# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '20px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

# Create plot
f.circle(x = 'Income', y = 'pct_increase', size=10,source=source)

#Create linear regression line
f.line(x="Income", y = "prediction",source=source,color="red")


# show plot
show(f)

# Speed vs. Non-vestern background

In [26]:
############ prepare data ##############################################################################################
df_temp = df_pct_ethnicity.copy()

df_ethnicity_speed = df_hast_change.copy()

df_ethnicity_speed["pct_not_vestern"] = df_temp["pct_not_vestern"]

df_ethnicity_speed = df_ethnicity_speed.reset_index()

df_ethnicity_speed.columns = ["Kommune","pct_increase","pct_not_vestern"]

#df_ethnicity_speed = df_ethnicity_speed[df_ethnicity_speed.Kommune != "Høje-Taastrup"]

#### Linear regression ###########################################################################################
X = df_ethnicity_speed.pct_not_vestern.to_numpy().reshape(-1,1)
Y = df_ethnicity_speed.pct_increase.to_numpy().reshape(-1,1)

linear_regressor = LinearRegression()

linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)

Y_pred = list(Y_pred.squeeze())

df_ethnicity_speed["prediction"] = Y_pred



########### Create figure #########################################################################################################
source = ColumnDataSource(data = df_ethnicity_speed)

TOOLTIPS = [("Municipality: ", "@Kommune"),
            ("% non-vestern", "@pct_not_vestern{%0.2f}"),
            ("% increase in speed: ","@pct_increase{%0.2f}")]

# create figure
f = figure(plot_width=400,plot_height=400,
    title = 'Increase in speed vs. % non-vestern',
           x_axis_label='% non-vestern',
           y_axis_label='% increase in speed',
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS,
           #y_axis_type="log"
          )

## Apply R^2 label ##############################################################################################

cor = df_ethnicity_speed[["pct_increase","pct_not_vestern"]].corr()

R_sqrt = round(cor.iloc[0,1]**2,3)


R_sqrt_Label = Label(x=160, y=290, x_units='screen', y_units='screen',
                 text=f'  R^2 value: {R_sqrt}  ', render_mode='canvas',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0,
                 text_font_size='14pt',text_font="times")

f.add_layout(R_sqrt_Label)

## plot #########################################################################################################

# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '20px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

# Create plot
f.circle(x = 'pct_not_vestern', y = 'pct_increase', size=10,source=source)

#Create linear regression line
f.line(x="pct_not_vestern", y = "prediction",source=source,color="red")


# show plot
show(f)